## Rotating information around a ring (non-blocking)

Now let’s repeat the exercise but you are now solving the deadlock in an optimal way using non-blocking communication.

* Substitute MPI_Send with MPI_Issend (non-blocking synchronous send) and put the wait statement at the correct place. Keep the normal blocking MPI_Recv. Run the program.

In [ ]:
?MPI::MPI_Issend

In [ ]:
?MPI::MPI_Wait

***
#### C program

In [ ]:
%%file ring.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank, size;
    int snd_buf, rcv_buf;
    int right, left;
    int sum, i;
    MPI_Status status;

    MPI_Init(NULL, NULL);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    right = (rank+1)      % size;
    left  = (rank-1+size) % size;

    sum = 0;
    snd_buf = rank;
    for(i = 0; i < size; i++) 
    {
        MPI_Send(&snd_buf, 1, MPI_INT, right, 17, MPI_COMM_WORLD);
        MPI_Recv(&rcv_buf, 1, MPI_INT, left,  17, MPI_COMM_WORLD, &status);
        snd_buf = rcv_buf;
        sum += rcv_buf;
    }
    printf ("PE%i:\tSum = %i\n", rank, sum);

    MPI_Finalize();
}

In [ ]:
!mpicc ring.c && mpirun -np 3 --allow-run-as-root a.out

***
#### Python program

In [ ]:
%%file ring.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

right = (rank+1) % size
left = (rank-1+size) % size
sum = 0
snd_buf = rank
rcv_buf = -1

for i in range (0,size):
    comm.send(snd_buf, dest=right)
    rcv_buf = comm.recv(source=left)
    snd_buf = rcv_buf
    sum += rcv_buf

print("PE%i:\tSum = %i" % (rank, sum))

In [ ]:
!mpirun -np 3 --allow-run-as-root python ring.py

***
#### Fortran program

In [ ]:
%%file ring.f90
program ring
use mpi

integer ( kind = 4 ) error
integer :: rank, size
integer :: right, left
integer :: i, sum
integer :: snd_buf, rcv_buf
integer :: status(MPI_STATUS_SIZE)

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)
right = mod(rank+1,      size)
left  = mod(rank-1+size, size)
sum = 0
snd_buf = rank
do i = 1, size
    call MPI_Send(snd_buf, 1, MPI_INTEGER, right, 17, MPI_COMM_WORLD, error)
    call MPI_Recv(rcv_buf, 1, MPI_INTEGER, left, 17, MPI_COMM_WORLD, status, error)
    snd_buf = rcv_buf
    sum = sum + rcv_buf
end do
print *, 'PE', rank, ': Sum =', sum
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 3 --allow-run-as-root a.out

***
### You can compare with our solution:

***
#### C solution

In [ ]:
%%file ring.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank, size;
    int snd_buf, rcv_buf;
    int right, left;
    int sum, i;
    MPI_Status status;
    MPI_Request request; //added request variable

    MPI_Init(NULL, NULL);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    right = (rank+1)      % size;
    left  = (rank-1+size) % size;

    sum = 0;
    snd_buf = rank;
    for(i = 0; i < size; i++) 
    {
        MPI_Issend(&snd_buf, 1, MPI_INT, right, 17, MPI_COMM_WORLD, &request); //substitute send with issend
        MPI_Recv(&rcv_buf, 1, MPI_INT, left,  17, MPI_COMM_WORLD, &status);
        MPI_Wait(&request, &status); //wait right after receive
        //only after this wait you are allowed to modify the send buffer
        //you must not modify the buffer between the issend and the wait
        snd_buf = rcv_buf;
        sum += rcv_buf;
    }
    printf ("PE%i:\tSum = %i\n", rank, sum);

    MPI_Finalize();
}

In [ ]:
!mpicc ring.c && mpirun -np 3 --allow-run-as-root a.out

***
#### Python solution

In [ ]:
%%file ring.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

right = (rank+1) % size
left = (rank-1+size) % size
sum = 0
snd_buf = rank
rcv_buf = -1

for i in range (0,size):
    request = comm.issend(snd_buf, dest=right)
    rcv_buf = comm.recv(source=left)
    request.wait()
    snd_buf = rcv_buf
    sum += rcv_buf

print("PE%i:\tSum = %i" % (rank, sum))

In [ ]:
!mpirun -np 3 --allow-run-as-root python ring.py

***
#### Fortran solution

In [ ]:
%%file ring.f90
program ring
use mpi

integer ( kind = 4 ) error
integer :: rank, size
integer :: right, left
integer :: i, sum
integer, asynchronous ::snd_buf ! additional declaration
integer :: rcv_buf
integer :: status(MPI_STATUS_SIZE)
integer :: request

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)
right = mod(rank+1,      size)
left  = mod(rank-1+size, size)
sum = 0
snd_buf = rank
do i = 1, size
    call MPI_Issend(snd_buf, 1, MPI_INTEGER, right, 17, MPI_COMM_WORLD, request, error)
    call MPI_Recv(rcv_buf, 1, MPI_INTEGER, left, 17, MPI_COMM_WORLD, status, error)
    call MPI_Wait(request, status, error)
    if (.NOT.MPI_ASYNC_PROTECTS_NONBLOCKING) call MPI_F_sync_reg(snd_buf) ! additional special statement
    snd_buf = rcv_buf
    sum = sum + rcv_buf
end do
print *, 'PE', rank, ': Sum =', sum
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 3 --allow-run-as-root a.out